In [2]:
!pip install openai==0.28.1
!pip install tiktoken==0.6.0
!pip install langchain==0.1.20
!pip install chromadb==0.5.0
!pip install pyPDF2
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


In [3]:
import openai
import numpy as np
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding
import faiss
import warnings
import os
import PyPDF2
warnings.filterwarnings("ignore")

In [4]:
os.environ['OPENAI_API_KEY'] = ''

In [5]:
openai.api_key = ''

In [6]:
def extract_text_from_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        for page in reader.pages:
            text += page.extract_text()
    return text

In [7]:
 pdf_text = extract_text_from_pdf('/financial-wellness-journal-english.pdf')

In [8]:
print(pdf_text)

Financial Wellness Journal 
Introduction
Getting started Pre-assessmentTABLE OF CONTENTS
Chapter 1: All About Financial Wellness
Chapter 
introductionHow FinEd can 
help you achieve 
financial wellnessThe life stages
of money Facing these 
difficulties head onActivity: Visioning 
exercise
Chapter 2: All About Saving
Case study: Mang 
Rafael’s saving 
problem Common 
misconceptions 
about savingWhat are the goals 
of saving?What are the tools 
we can use to start 
saving?Where do we save?
Case study: Aling 
Yolanda insures her 
daughter’s future What is insurance? Why do we need 
to insure ourselves 
and our family?What are the 
different kinds
of insurance?Chapter wrap-up
Chapter 4: All About CreditChapter 3: All About Insurance
Chapter 5: All About Investing
Conclusion3 4
6 7 8 9 11
12 13 14 15 20
23 21 24
27 28 29 29 31
33 34 35 36
38 39 39 4037
41 42 43 43 46Where should our 
savings go?What are the 
methods of 
budgeting?Chapter wrap-up
Case study: 
Teacher Mary is 
finally debt-fr

In [9]:
print(len(pdf_text))

80030


In [35]:
chunk_size = 5000
chunks = [pdf_text[i:i + chunk_size] for i in range(0, len(pdf_text), chunk_size)]

print(chunks)

['Financial Wellness Journal \nIntroduction\nGetting started Pre-assessmentTABLE OF CONTENTS\nChapter 1: All About Financial Wellness\nChapter \nintroductionHow FinEd can \nhelp you achieve \nfinancial wellnessThe life stages\nof money Facing these \ndifficulties head onActivity: Visioning \nexercise\nChapter 2: All About Saving\nCase study: Mang \nRafael’s saving \nproblem Common \nmisconceptions \nabout savingWhat are the goals \nof saving?What are the tools \nwe can use to start \nsaving?Where do we save?\nCase study: Aling \nYolanda insures her \ndaughter’s future What is insurance? Why do we need \nto insure ourselves \nand our family?What are the \ndifferent kinds\nof insurance?Chapter wrap-up\nChapter 4: All About CreditChapter 3: All About Insurance\nChapter 5: All About Investing\nConclusion3 4\n6 7 8 9 11\n12 13 14 15 20\n23 21 24\n27 28 29 29 31\n33 34 35 36\n38 39 39 4037\n41 42 43 43 46Where should our \nsavings go?What are the \nmethods of \nbudgeting?Chapter wrap-up\nCas

In [36]:
embeddings = [get_embedding(chunk, engine = "text-embedding-3-small") for chunk in chunks]

In [37]:
embedding_dim = len(embeddings[0])

In [38]:
embeddings_np = np.array(embeddings).astype('float32')

In [39]:
index = faiss.IndexFlatL2(embedding_dim)

In [40]:
index.add(embeddings_np)

In [41]:
user_message = "How can i be a good investor?"

In [42]:
query_embedding = get_embedding(user_message, engine='text-embedding-3-small')
query_embedding_np = np.array([query_embedding]).astype('float32')

In [43]:
_, indices = index.search(query_embedding_np, 5)

In [44]:
retrieved_chunks = [chunks[i] for i in indices[0]]

In [45]:
context = ' '.join(retrieved_chunks)

In [46]:
structured_prompt = f"Context:\n{context}\n\nQuery:\n{user_message}\n\nResponse:"

In [47]:
print(structured_prompt)

Context:
deposit interest rate of 2% (perhaps a time deposit 
account), and who leave the money account), and who leave the money 
untouched until they turn 60 years old. untouched until they turn 60 years old. 
As you can see, Anna, who invested the As you can see, Anna, who invested the 
earliest, earned the most among the three earliest, earned the most among the three 
friends. Benjie, earned less than Anna friends. Benjie, earned less than Anna 
but still earned more than Charlie, who but still earned more than Charlie, who 
invested the latest. Charlie, though he invested the latest. Charlie, though he invested late, still got a good return for his invested late, still got a good return for his 
investment, though not as much as his investment, though not as much as his 
other friends. other friends. 
This just shows thatThis just shows that   the earlier you the earlier you 
invest, the more interest your invest, the more interest your 
investment will earn over time and the inv

In [48]:
struct = [{"role": "system", "content":"You are a helpful Assitant. Mention if its not in the scope"}]

In [49]:
chat =  openai.ChatCompletion.create(model = "gpt-4o-mini", messages = struct + [{"role": "user", "content" : structured_prompt}], temperature=0.5, max_tokens=1500, top_p=1, frequency_penalty=0, presence_penalty=0)
struct.append({"role": "user", "content": user_message})
response = chat.choices[0].message.content
struct.append({"role": "assistant", "content": response})

In [50]:
print(response)

To become a good investor, consider the following steps:

1. **Educate Yourself**: Understand the basics of investing, including different types of investment vehicles (stocks, bonds, mutual funds, etc.), market dynamics, and financial principles. Read books, take courses, and follow reputable financial news sources.

2. **Set Clear Goals**: Define your investment objectives. Are you saving for retirement, a home, or education? Knowing your goals will help you choose the right investment strategy.

3. **Know Your Risk Tolerance**: Assess your risk appetite based on factors like your age, income, financial responsibilities, and comfort with market fluctuations. This will guide you in selecting appropriate investment options.

4. **Diversify Your Portfolio**: Spread your investments across various asset classes to reduce risk. Diversification helps protect your portfolio from significant losses if one investment performs poorly.

5. **Start Early and Invest Regularly**: Take advantage of